In [ ]:
import pandas as pd
import numpy as np
import torch

from proprietary_data import CompanyFundamentalsKind, get_data_frame
from scipy import stats

from torchmetrics import MetricCollection
from torchmetrics import (
    MeanAbsoluteError,
    MeanSquaredError,
    R2Score,
    MeanAbsolutePercentageError,
    SymmetricMeanAbsolutePercentageError,
    RelativeSquaredError,
)

In [ ]:
df = get_data_frame(CompanyFundamentalsKind.Normalized, subset=False, min_length="max")
df

In [ ]:
human_column = "Revenue F12M Analyst Estimate"

In [ ]:
df_key = df[df[human_column].notna()]
f"{100*df_key.size / df.size:.2f}% of the data has expert estimates"

In [ ]:
# np.savetxt(
#     "human_eval_companies.csv", df_key["companyid"].unique(), delimiter=",", fmt="%s"
# )

In [ ]:
df_key = df_key[
    # For some reason, this is the date range of the expert estimates
    (df_key["aca_quarter"] >= "2013-07-01") & (df_key["aca_quarter"] <= "2022-04-01")
]
df_key

In [ ]:
from acatis_data import get_data_transform

data_transform = get_data_transform(CompanyFundamentalsKind.Normalized)

In [ ]:
mean_rev, mean_est = data_transform.standardizer.mean_[
    [
        data_transform._feature_names.index("Total Revenues"),
        data_transform._feature_names.index("Revenue F12M Analyst Estimate"),
    ]
]
mean_rev, mean_est

In [ ]:
var_rev, var_est = data_transform.standardizer.var_[
    [
        data_transform._feature_names.index("Total Revenues"),
        data_transform._feature_names.index("Revenue F12M Analyst Estimate"),
    ]
]
var_rev, var_est

In [ ]:
(df_key["Revenue F12M Analyst Estimate"] * var_est).var()

In [ ]:
df_key_corrected = df_key.copy()
df_key_corrected["Revenue F12M Analyst Estimate"] = (
    df_key_corrected["Revenue F12M Analyst Estimate"] * var_est + mean_est - mean_rev
) # / var_rev

In [ ]:
specific = df_key_corrected[df_key_corrected["companyname"] == "Cisco Systems, Inc."]
specific.plot(
    x="aca_quarter",
    y=["Total Revenues", "Revenue F12M Analyst Estimate"],
    title="ABB Ltd",
)
pass

In [ ]:
# Not all have quite the same size
df = df_key_corrected
sections = [
    df[df["aca_quarter"] == quarter][["Total Revenues", human_column]]
    for quarter in df["aca_quarter"].unique()
]
len(sections)

In [ ]:
# truth, estimate = df_key[["Total Revenues", human_column]].to_numpy().T
# truth.shape, estimate.shape

In [ ]:
metrics = MetricCollection(
    {
        "MAE": MeanAbsoluteError(),
        "MSE": MeanSquaredError(),
        "RMSE": MeanSquaredError(squared=False),
        "R2": R2Score(),
        "MAPE": MeanAbsolutePercentageError(),
        "SMAPE": SymmetricMeanAbsolutePercentageError(),
        "RSE": RelativeSquaredError(),
    }
)

lim = 4

In [ ]:
def mae(pred: np.ndarray, truth: np.ndarray) -> np.ndarray:
    res = np.abs(pred - truth)
    res = res[stats.zscore(res) < lim]
    return res.mean()

def mse(pred: np.ndarray, truth: np.ndarray) -> np.ndarray:
    res = np.square(pred - truth)
    res = res[stats.zscore(res) < lim]
    return res.mean()

def rmse(pred: np.ndarray, truth: np.ndarray) -> np.ndarray:
    overall = mse(pred, truth)
    return np.sqrt(overall.mean())

def mape(
    pred: np.ndarray, truth: np.ndarray, epsilon: float = 1.17e-06
) -> np.ndarray:
    res = np.abs(pred - truth) / np.maximum(np.abs(truth), epsilon)
    res = res[stats.zscore(res) < lim]
    return res.mean()

def rse(
    pred: np.ndarray, truth: np.ndarray, epsilon: float = 1.17e-06
) -> np.ndarray:
    divisor = np.square(truth - truth.mean())
    # np.square(truth - truth.mean(fh_dim), fh_dim)) + epsilon
    above = np.square(pred - truth)
    mask = np.logical_and(stats.zscore(divisor) < lim, stats.zscore(above) < lim)
    return above[mask].mean() / (divisor[mask] + epsilon).mean()

def smape(
    pred: np.ndarray, truth: np.ndarray, epsilon: float = 1.17e-06
) -> np.ndarray:
    res = np.abs(pred - truth) / np.maximum(
        np.abs(pred), np.maximum(np.abs(truth), epsilon)
    )
    res = res[stats.zscore(res) < lim]
    return res.mean()

def r2(
    pred: np.ndarray, truth: np.ndarray, epsilon: float = 1.17e-06
) -> np.ndarray:
    # mean over horizon
    quadratic_error = np.square(pred - truth)
    mask = stats.zscore(quadratic_error) < lim
    result = np.sum(quadratic_error[mask]) / (
        np.sum(np.square(truth - truth.mean())[mask])
        + epsilon
    )
    return 1 - result

def compute_all(preds: np.ndarray, targets: np.ndarray):
    return {
        "MAE": mae(preds, targets),
        "MSE": mse(preds, targets),
        "RMSE": rmse(preds, targets),
        "MAPE": mape(preds, targets),
        "RSE": rse(preds, targets),
        "SMAPE": smape(preds, targets),
        "R2": r2(preds, targets),
    }

In [ ]:
errors = [
    {"split": split, "metric": name, "value": value.item()}
    for split, section in enumerate(sections)
    for name, value in compute_all(
        section["Total Revenues"].to_numpy(),
        section[human_column].to_numpy(),
    ).items()
]
len(errors)

In [ ]:
pd.DataFrame(errors)

In [ ]:
df = (
    pd.DataFrame(errors)
    .groupby("metric")
    .agg({"value": ["mean", "std"]})
    .droplevel(0, axis=1)
)

df["Human Analysts"] = (
    df["mean"].apply(lambda x: f"{x:0>2.3f}")
    + "±"
    + df["std"].apply(lambda x: f"{x:.2f}")
)
del df["mean"]
del df["std"]

as_col = df.transpose()
as_col["nCRPS"] = as_col["MAE"]

print(
    as_col[
        [
            "MAE",
            "MSE",
            "RMSE",
            "MAPE",
            "RSE",
            "SMAPE",
            "R2",
            "nCRPS",
        ]
    ].to_latex()
)